In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import os 
import pickle
from collections import Counter
from pathlib import Path
import csv
import multiprocessing as mp
import re
import ast

In [2]:
markers = ['FSC-A','FSC-H','FSC-W','SSC-A','SSC-H','SSC-W','PSTAT1','PSTAT5','CD66B','CD33','CD3/CD14','PSTAT3','CD4/CD19']
row_cutoff = int(2e4)

In [3]:
mfest = pd.read_csv('fcs_metadata.csv')
mfest['markers'] = mfest.apply(lambda row: ast.literal_eval(row.markers), axis=1)
mfest['fcs'] = mfest.apply(lambda row: Path(row.fcs), axis=1)
mfest['pkl'] = mfest.apply(lambda row: Path(row.pkl), axis=1)
mfest

,Unnamed: 0,Subject Accession,Species,Race,Race Specify,Ethnicity,Strain,Gender,Age Event,Age Event Specify,...,File Info ID,File Detail,File Name,Original File Name,fcs,pkl,pkl_exists,n_rows,n_columns,markers
0,0,SUB137174,Homo sapiens,Black or African American,NaN,Not Hispanic or Latino,NaN,Female,Age at enrollment,NaN,...,532468,Flow cytometry result,RC4_080513_RC4_080513_11-020_IFNa_B04.532468.fcs,RC4_080513_RC4_080513_11-020_IFNa_B04.fcs,/home/ubuntu/a/aging/data/ResultFiles/Flow_cyt...,/home/ubuntu/a/aging/data/ResultFiles/pickles/...,True,41826.0,13.0,"[FSC-A, FSC-H, FSC-W, SSC-A, SSC-H, SSC-W, PST..."
1,1,SUB137174,Homo sapiens,Black or African American,NaN,Not Hispanic or Latino,NaN,Female,Age at enrollment,NaN,...,532469,Flow cytometry result,RC4_080513_RC4_080513_11-020_IL10_D04.532469.fcs,RC4_080513_RC4_080513_11-020_IL10_D04.fcs,/home/ubuntu/a/aging/data/ResultFiles/Flow_cyt...,/home/ubuntu/a/aging/data/ResultFiles/pickles/...,True,41872.0,13.0,"[FSC-A, FSC-H, FSC-W, SSC-A, SSC-H, SSC-W, PST..."
2,2,SUB137174,Homo sapiens,Black or African American,NaN,Not Hispanic or Latino,NaN,Female,Age at enrollment,NaN,...,532470,Flow cytometry result,RC4_080513_RC4_080513_11-020_IL21_E04.532470.fcs,RC4_080513_RC4_080513_11-020_IL21_E04.fcs,/home/ubuntu/a/aging/data/ResultFiles/Flow_cyt...,/home/ubuntu/a/aging/data/ResultFiles/pickles/...,True,30830.0,13.0,"[FSC-A, FSC-H, FSC-W, SSC-A, SSC-H, SSC-W, PST..."
3,3,SUB137174,Homo sapiens,Black or African American,NaN,Not Hispanic or Latino,NaN,Female,Age at enrollment,NaN,...,532471,Flow cytometry result,RC4_080513_RC4_080513_11-020_IL6_C04.532471.fcs,RC4_080513_RC4_080513_11-020_IL6_C04.fcs,/home/ubuntu/a/aging/data/ResultFiles/Flow_cyt...,/home/ubuntu/a/aging/data/ResultFiles/pickles/...,True,49194.0,13.0,"[FSC-A, FSC-H, FSC-W, SSC-A, SSC-H, SSC-W, PST..."
4,4,SUB137174,Homo sapiens,Black or African American,NaN,Not Hispanic or Latino,NaN,Female,Age at enrollment,NaN,...,532472,Flow cytometry result,RC4_080513_RC4_080513_11-020_US_A04.532472.fcs,RC4_080513_RC4_080513_11-020_US_A04.fcs,/home/ubuntu/a/aging/data/ResultFiles/Flow_cyt...,/home/ubuntu/a/aging/data/ResultFiles/pickles/...,True,74924.0,13.0,"[FSC-A, FSC-H, FSC-W, SSC-A, SSC-H, SSC-W, PST..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125,1125,SUB147387,Homo sapiens,Not Specified,NaN,Not Specified,NaN,Not Specified,Not Specified,NaN,...,533134,Flow cytometry result,s_3-control-2.533134.fcs,s_3-control-2.fcs,/home/ubuntu/a/aging/data/ResultFiles/Flow_cyt...,/home/ubuntu/a/aging/data/ResultFiles/pickles/...,True,500000.0,15.0,"[FSC-A, FSC-H, FSC-W, SSC-A, SSC-H, SSC-W, CD6..."
1126,1126,SUB147387,Homo sapiens,Not Specified,NaN,Not Specified,NaN,Not Specified,Not Specified,NaN,...,533134,Flow cytometry result,s_3-control-2.533134.fcs,s_3-control-2.fcs,/home/ubuntu/a/aging/data/ResultFiles/Flow_cyt...,/home/ubuntu/a/aging/data/ResultFiles/pickles/...,True,500000.0,15.0,"[FSC-A, FSC-H, FSC-W, SSC-A, SSC-H, SSC-W, CD6..."
1127,1127,SUB147387,Homo sapiens,Not Specified,NaN,Not Specified,NaN,Not Specified,Not Specified,NaN,...,533134,Flow cytometry result,s_3-control-2.533134.fcs,s_3-control-2.fcs,/home/ubuntu/a/aging/data/ResultFiles/Flow_cyt...,/home/ubuntu/a/aging/data/ResultFiles/pickles/...,True,500000.0,15.0,"[FSC-A, FSC-H, FSC-W, SSC-A, SSC-H, SSC-W, CD6..."
1128,1128,SUB147387,Homo sapiens,Not Specified,NaN,Not Specified,NaN,Not Specified,Not Specified,NaN,...,533134,Flow cytometry result,s_3-control-2.533134.fcs,s_3-control-2.fcs,/home/ubuntu/a/aging/data/ResultFiles/Flow_cyt...,/home/ubuntu/a/aging/data/ResultFiles/pickles/...,True,500000.0,15.0,"[FSC-A, FSC-H, FSC-W, SSC-A, SSC-H, SSC-W, CD6..."


In [4]:
def arcsinh(x):
    return(np.arcsinh(x/5))

In [5]:
marker_counts = Counter()
for markers in mfest['markers']:
    marker_counts.update(markers)
marker_counts

Counter({'FSC-A': 1130,
         'FSC-H': 1130,
         'FSC-W': 1130,
         'SSC-A': 1130,
         'SSC-H': 1130,
         'SSC-W': 1130,
         'PSTAT1': 819,
         'PSTAT5': 819,
         'CD66B': 1114,
         'CD33': 1114,
         'CD3/CD14': 874,
         'PSTAT3': 819,
         'CD4/CD19': 1114,
         'PSTAT-1': 295,
         'PSTAT-5': 295,
         'CD14/CD3': 240,
         'PSTAT-3': 295,
         'APC-ALEXA 750-A': 560,
         'PACIFIC ORANGE-A': 560,
         'FITC-A': 16,
         'PE-A': 16,
         'PERCP-CY5-5-A': 16,
         'PE-CY7-A': 16,
         'PACIFIC BLUE-A': 16,
         'APC-A': 16,
         'ALEXA 700-A': 16})

In [6]:
def filter_by_markers(df, q_markers):
    q_markers = set(q_markers)
    return df[df.apply(
        lambda row: len(q_markers & set(row['markers'])) == len(q_markers),
    axis=1
    )]   
markers = ['FSC-A','FSC-H','FSC-W','SSC-A','SSC-H','SSC-W','PSTAT1','PSTAT5','CD66B','CD33','CD3/CD14','PSTAT3','CD4/CD19']
samples = filter_by_markers(mfest, markers)
samples = samples[samples.n_rows>row_cutoff]
samples.shape

(753, 47)

In [7]:
len(markers)

13

In [8]:
samples.n_rows.describe()

count       753.000000
mean     268850.689243
std      193124.336256
min       20079.000000
25%       96002.000000
50%      196825.000000
75%      500000.000000
max      771997.000000
Name: n_rows, dtype: float64

In [9]:
Counter(samples['Expsample Treatement'])

Counter({'interferon alpha': 150,
         'IL-10': 154,
         'IL-21': 149,
         'IL-6': 151,
         'Unstim': 149})

In [10]:
pd.read_pickle(samples.iloc[0].pkl)

,FSC-A,FSC-H,FSC-W,SSC-A,SSC-H,SSC-W,PSTAT1,PSTAT5,CD66B,CD33,CD3/CD14,PSTAT3,CD4/CD19
1,105764.164062,74308.0,93278.781250,262143.000000,256592.0,66953.781250,506.679962,-4.240000,23.320000,1240.199951,1085.800049,1535.380005,70.070000
2,77193.359375,67022.0,75481.835938,137303.906250,95466.0,94257.109375,1093.919922,799.239929,558.619995,287.259979,1178.520020,363.440002,560.559998
3,55635.628906,51456.0,70859.312500,50348.937500,41268.0,79957.054688,407.039978,463.219971,612.679993,505.619965,85.400002,127.820000,315.699982
4,58125.570312,52400.0,72696.898438,63292.597656,48224.0,86014.093750,753.659973,481.239960,9193.379883,2867.299805,1163.270020,645.260010,5482.399902
5,69993.242188,64830.0,70755.468750,60593.835938,52974.0,74962.765625,778.039978,470.639984,7899.119629,3538.279785,1026.020020,546.700012,4466.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41822,64154.671875,57778.0,72768.882812,47112.757812,41174.0,74988.625000,587.239990,354.039978,13847.838867,5809.859863,871.080017,667.589966,3037.649902
41823,262143.000000,256280.0,67035.289062,262143.000000,256684.0,66929.781250,10040.319336,6278.379883,7396.679688,5583.019531,13742.690430,4083.309814,6661.270020
41824,262143.000000,152453.0,112689.179688,262143.000000,126577.0,135726.109375,2993.439941,2092.439941,2888.499756,1129.959961,4653.689941,1021.789978,2018.939941
41825,93842.546875,68284.0,90065.968750,59147.996094,40268.0,96263.109375,762.139954,561.799988,753.659973,220.479980,1664.080078,248.709991,519.750000


In [11]:
def read_frame(pkl):
    df = pd.read_pickle(pkl)
    df = df[markers]
    df = df.iloc[:int(row_cutoff),:]
    df = df.apply(arcsinh)
    new_shape = list(df.shape)+[1]
    d = df.values
    d = d.reshape(new_shape)
    print(pkl.name)
    return d

In [12]:
pool = mp.Pool()

RC4-031114_RC4-031114_11-0257_IL10_D02.532474.pklRC4_080613_RC4_080613_RC4-080613_RC4_080613_11-055_IFNa_B01.532572.pklRC4_080513_RC4_080513_11-020_IFNa_B04.532468.pkl

RC4_080613_RC4_080613_RC4-080613_RC4_080613_11-058_IL6_C03.532615.pkl

RC4-031114_RC4-031114_11-0151_US_A03.532462.pklRC4_080513_RC4_080513_11-043_IL21_E06.532510.pkl

RC4-080813_weiqiupdate_RC4-080813_weiqiupdate_11-120_IFNa_B05.532866.pkl
RC4-031114_RC4-031114_11-0257_IL21_E02.532475.pkl
RC4_080613_RC4_080613_RC4-080613_RC4_080613_11-055_IL10_D01.532573.pklRC4_080613_RC4_080613_RC4-080613_RC4_080613_11-080_IL6_C06.532834.pkl

RC4_080513_RC4_080513_11-020_IL10_D04.532469.pkl
RC4_080613_RC4_080613_RC4-080613_RC4_080613_11-058_US_A03.532616.pkl
RC4_080513_RC4_080513_11-272_IFNa_B01.532893.pklRC4-080813_weiqiupdate_RC4-080813_weiqiupdate_11-120_IL10_D05.532867.pkl

RC4_072213_RC4_072213_RC4_RC4-072213-1_RC4_072213_11-274_IL21_E05.532905.pkl
RC4_080613_RC4_080613_RC4-080613_RC4_080613_11-055_IL21_E01.532574.pklRC4_080513_R

In [13]:
expr_list = pool.map(read_frame, samples.pkl)
expr_list = np.stack(expr_list)
print("The dimenstion of the data is: ", expr_list.shape)

The dimenstion of the data is:  (753, 20000, 13, 1)


In [14]:
pool.close()
pool.terminate()

In [15]:
allData = {
    'samples': samples,
    'expr_list': expr_list,
    'marker_names': pd.Series(markers)
}
with open('agingData.obj','wb') as wf:
    pickle.dump(allData, wf)